In [1]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            #print(action['class'] + '_' + str(action['id']),static[str(action['class']) + '_' + str(action['id'])]['damage'])
                #print(action)
            if 'damage' in action:
                for damage in action['damage']:
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[2][0])                         
                    #['ATK', ['monster', 616666], [108], [108], ['Equal'], ['Default']]  [108] 是伤害值
            if 'effects' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['effects']=[]

                        
#             if action['class'] + '_' + str(action['id'])=='monster_716666':
#                 print('77777', action['class'] + '_' + str(action['id']))
            
                
            if 'effects' in action:
                if len(action['effects'])>0:
                    for r in action['effects']:
                        static[str(action['class']) + '_' + str(action['id'])]['effects'].append(r[1])
        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]

    for k in static.keys():
        if 'damage' not in static[k]:
            static[k]['damage']=[]
        if 'effects' not in static[k]:
            static[k]['effects']=[]
        #print(k,static[k])
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        static[k]['各类效果']= list(set(num for sublist in static[k]['effects'] for num in sublist))
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']
        
        
        #隐藏具体伤害
        static[k].pop('damage')
        static[k].pop('effects')

        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    print('总tick',res[-1]['tick'])
    return static


In [2]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T
        display(df)

总tick 648


,BOTTOM,LEFT,MOVE_START,RIGHT,SKILL_133,TOP,各类效果,平均伤害,总伤害,总伤害次数,总移动次数,最终HP,WAIT,SKILL_135,SKILL_136,SKILL_130,SKILL_131,SKILL_132,SKILL_87,SKILL_88,SKILL_90,SKILL_138,EFFECT_64,SKILL_100,SKILL_96,SKILL_77,SKILL_78,SKILL_80,SKILL_137
monster_616666,5,2,5,6,2,1,[78],0.0,0,2,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_1316666,1,8,15,3,2,7,[],0.0,0,2,19,0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_7115666,2,5,27,6,6,4,[78],0.0,0,6,17,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_8115666,8,1,29,3,NaN,0,"[96, 65, 66]",0.0,0,6,12,217,22,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_16714666,0,3,29,6,NaN,4,"[96, 65, 66, 67]",0.0,0,10,13,0,16,NaN,NaN,5,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5003,3,4,23,8,NaN,17,[],105.52,2638,25,32,NaN,2,NaN,NaN,NaN,NaN,NaN,11,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_516666,0,0,2,3,NaN,0,[],0.0,0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_716666,1,1,2,3,NaN,0,[],0.0,0,1,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5001,0,0,19,12,NaN,13,[],60.210526,1144,19,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,4,14,NaN,NaN,NaN,NaN
hero_5002,1,5,18,19,NaN,15,[],69.277778,1247,18,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,9,2,5,NaN


In [32]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            if actions[i]['id']==16714666:
                if actions[i]['action_type']!='MOVE_START' and actions[i]['action_type']!='WAIT':
                    print(actions[i],states[i],r['tick'])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(99)+".json"
log_path = log_files_path + "/for_qiangye.json"

parse_result_json_detail(log_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lhwylp/Desktop/publicwork/game_dt_demo/V1/test/test_data/for_qiangye.json'

In [12]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'hero' in v:
                if v['hero']==1216666:
                    if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                        if  v['stepname']=='决策树-首次查找最终选择' or  v['stepname']=='决策树-多次查找最终选择':
                            #if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                                print(k,v)
                        
def static_decision_log(log_path):
    static={}
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if  v['stepname']=='决策树-首次查找最终选择' and v['result_len']!=0:
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                    
                if  v['stepname']=='决策树-多次查找最终选择':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                if v['stepname']=='决策树-偏好决策':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']]['偏好_'+str(v['action'])]=1
                    static[v['hero']]['偏好_'+str(v['action'])]=static[v['hero']]['偏好_'+str(v['action'])]+1

    df=pd.DataFrame(static).T
    display(df)

In [13]:
#统计情况
static_decision_log(os.getcwd()+"/../log/e864e9e1792024_11_04_16_05_1730707558logs.json")


,偏好_等待,移动到敌人附近,移动到敌人附近并攻击,逃跑,等待,偏好_p_0_action,移动到boss
616666,2.0,2.0,4.0,2.0,NaN,NaN,NaN
1316666,15.0,2.0,4.0,4.0,8.0,NaN,NaN
7115666,30.0,3.0,10.0,7.0,13.0,NaN,NaN
8115666,30.0,2.0,6.0,5.0,20.0,NaN,NaN
16714666,30.0,3.0,9.0,5.0,16.0,NaN,NaN
5003,2.0,NaN,5.0,NaN,NaN,2.0,NaN
5001,2.0,NaN,18.0,NaN,NaN,NaN,4.0
5002,2.0,NaN,15.0,NaN,NaN,NaN,5.0
516666,2.0,NaN,2.0,2.0,NaN,NaN,NaN
716666,2.0,NaN,2.0,2.0,NaN,NaN,NaN


In [15]:

tmp_log_file = os.getcwd()+"/../log/e864e9e1792024_11_04_16_05_1730707558logs.json"
parse_decision_log(tmp_log_file)

2024-11-04 16:06:02.605288_164 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 1216666}
2024-11-04 16:06:12.983813_465 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 1216666}
2024-11-04 16:06:26.193245_737 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 1216666}
2024-11-04 16:06:39.853002_932 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '移动到敌人附近并攻击', 'hero': 1216666}
2024-11-04 16:07:00.153222_1181 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑', 'hero': 1216666}
2024-11-04 16:07:23.500797_1432 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑', 'hero': 1216666}
2024-11-04 16:07:44.262887_1622 {'stepname': '决策树-首次查找最终选择', 'result_len': 2, 'action': '移动到敌人附近并攻击', 'hero': 1216666}
2024-11-04 16:08:13.479880_1863 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '移动到敌人附近并攻击', 'hero': 1216666}
2024-11-04 16:08:44.015054_2098 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'